In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
#모든 열 보기
pd.set_option('display.max_columns', None)

In [3]:
path = 'D:/Dataset/Competitons/Jeju/'
train = pd.read_csv(path + 'train.csv', parse_dates=["base_date"]).copy()
test = pd.read_csv(path + 'test.csv', parse_dates=["base_date"]).copy()
origin_train = train.copy()
sample_submission = pd.read_csv(path + '/sample_submission.csv').copy()
train

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,weight_restricted,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,2022-06-23,목,17,0,1,106,지방도1112호선,0,0,60.0,0.0,32400.0,0.0,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,2022-07-28,목,21,0,2,103,일반국도11호선,0,0,60.0,0.0,0.0,0.0,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,2021-10-10,일,7,0,2,103,일반국도16호선,0,0,80.0,0.0,0.0,0.0,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,2022-03-11,금,13,0,2,107,태평로,0,0,50.0,0.0,0.0,0.0,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,2021-10-05,화,8,0,2,103,일반국도12호선,0,0,80.0,0.0,0.0,0.0,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,TRAIN_4701212,2021-11-04,목,16,0,1,107,-,0,0,50.0,0.0,0.0,0.0,0,대림사거리,33.422145,126.278125,없음,금덕해운,33.420955,126.273750,없음,20.0
4701213,TRAIN_4701213,2022-03-31,목,2,0,2,107,-,0,0,80.0,0.0,43200.0,0.0,3,광삼교,33.472505,126.424368,없음,광삼교,33.472525,126.424890,없음,65.0
4701214,TRAIN_4701214,2022-06-13,월,22,0,2,103,일반국도12호선,0,0,60.0,0.0,0.0,0.0,0,고성교차로,33.447183,126.912579,없음,성산교차로,33.444121,126.912948,없음,30.0
4701215,TRAIN_4701215,2021-10-20,수,2,0,2,103,일반국도95호선,0,0,80.0,0.0,0.0,0.0,0,제6광령교,33.443596,126.431817,없음,관광대학입구,33.444996,126.433332,없음,73.0


변수명,변수 설명
1. id,아이디
2. base_date,날짜
3. day_of_week,요일
4. base_hour,시간대
5. road_in_use,도로사용여부
6. lane_count,차로수
7. road_rating,도로등급
8. multi_linked,중용구간 여부
9. connect_code,연결로 코드
10. maximum_speed_limit,최고속도제한
11. weight_restricted,통과제한하중
12. hight_restricted,통과제한높이
13. road_type,도로유형
14. start_latitude,시작지점의 위도
15. start_longitude,시작지점의 경도
16. start_turn_restricted,시작 지점의 회전제한 유무
17. end_latitude,도착지점의 위도
18. end_longitude,도착지점의 경도
19. end_turn_restricted,도작지점의 회전제한 유무
20. road_name,도로명
21. start_node_name,시작지점명
22. end_node_name,도착지점명
23. vehicle_restricted,통과제한차량
24. target,평균속도(km)

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701217 entries, 0 to 4701216
Data columns (total 24 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   id                     object        
 1   base_date              datetime64[ns]
 2   day_of_week            object        
 3   base_hour              int64         
 4   road_in_use            int64         
 5   lane_count             int64         
 6   road_rating            int64         
 7   road_name              object        
 8   multi_linked           int64         
 9   connect_code           int64         
 10  maximum_speed_limit    float64       
 11  vehicle_restricted     float64       
 12  weight_restricted      float64       
 13  height_restricted      float64       
 14  road_type              int64         
 15  start_node_name        object        
 16  start_latitude         float64       
 17  start_longitude        float64       
 18  start_turn_restricted 

In [5]:
train.head(5)

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,weight_restricted,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,2022-06-23,목,17,0,1,106,지방도1112호선,0,0,60.0,0.0,32400.0,0.0,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,2022-07-28,목,21,0,2,103,일반국도11호선,0,0,60.0,0.0,0.0,0.0,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,2021-10-10,일,7,0,2,103,일반국도16호선,0,0,80.0,0.0,0.0,0.0,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,2022-03-11,금,13,0,2,107,태평로,0,0,50.0,0.0,0.0,0.0,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,2021-10-05,화,8,0,2,103,일반국도12호선,0,0,80.0,0.0,0.0,0.0,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0


In [7]:
from sklearn.preprocessing import LabelEncoder

def data_change(df):
#     df['day_of_week'] = LabelEncoder().fit_transform(df['day_of_week'])
#     df['road_name'] = LabelEncoder().fit_transform(df['road_name'])
#     df['start_node_name'] = LabelEncoder().fit_transform(df['start_node_name'])
#     df['start_turn_restricted'] = LabelEncoder().fit_transform(df['start_turn_restricted'])
#     df['end_node_name'] = LabelEncoder().fit_transform(df['end_node_name'])
#     df['end_turn_restricted'] = LabelEncoder().fit_transform(df['end_turn_restricted'])
    
    df['year'] = df['base_date'].dt.year
    df['month'] = df['base_date'].dt.month
    df['day'] = df['base_date'].dt.day
    df['quarter'] = df['base_date'].dt.quarter
    
    return df

train = data_change(train)
test = data_change(test)
train

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,weight_restricted,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,year,month,day,quarter
0,TRAIN_0000000,2022-06-23,목,17,0,1,106,지방도1112호선,0,0,60.0,0.0,32400.0,0.0,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0,2022,6,23,2
1,TRAIN_0000001,2022-07-28,목,21,0,2,103,일반국도11호선,0,0,60.0,0.0,0.0,0.0,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0,2022,7,28,3
2,TRAIN_0000002,2021-10-10,일,7,0,2,103,일반국도16호선,0,0,80.0,0.0,0.0,0.0,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0,2021,10,10,4
3,TRAIN_0000003,2022-03-11,금,13,0,2,107,태평로,0,0,50.0,0.0,0.0,0.0,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0,2022,3,11,1
4,TRAIN_0000004,2021-10-05,화,8,0,2,103,일반국도12호선,0,0,80.0,0.0,0.0,0.0,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0,2021,10,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,TRAIN_4701212,2021-11-04,목,16,0,1,107,-,0,0,50.0,0.0,0.0,0.0,0,대림사거리,33.422145,126.278125,없음,금덕해운,33.420955,126.273750,없음,20.0,2021,11,4,4
4701213,TRAIN_4701213,2022-03-31,목,2,0,2,107,-,0,0,80.0,0.0,43200.0,0.0,3,광삼교,33.472505,126.424368,없음,광삼교,33.472525,126.424890,없음,65.0,2022,3,31,1
4701214,TRAIN_4701214,2022-06-13,월,22,0,2,103,일반국도12호선,0,0,60.0,0.0,0.0,0.0,0,고성교차로,33.447183,126.912579,없음,성산교차로,33.444121,126.912948,없음,30.0,2022,6,13,2
4701215,TRAIN_4701215,2021-10-20,수,2,0,2,103,일반국도95호선,0,0,80.0,0.0,0.0,0.0,0,제6광령교,33.443596,126.431817,없음,관광대학입구,33.444996,126.433332,없음,73.0,2021,10,20,4


In [8]:
test

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,weight_restricted,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,year,month,day,quarter
0,TEST_000000,2022-08-25,목,17,0,3,107,연삼로,0,0,70.0,0.0,0.0,0.0,0,산지2교,33.499427,126.541298,없음,제주은행사거리,33.500772,126.543837,있음,2022,8,25,3
1,TEST_000001,2022-08-09,화,12,0,2,103,일반국도12호선,0,0,70.0,0.0,0.0,0.0,3,중문입구,33.258507,126.427003,없음,관광단지입구,33.258119,126.415840,없음,2022,8,9,3
2,TEST_000002,2022-08-05,금,2,0,1,103,일반국도16호선,0,0,60.0,0.0,0.0,0.0,0,도순3교,33.258960,126.476508,없음,도순2교,33.259206,126.474687,없음,2022,8,5,3
3,TEST_000003,2022-08-18,목,23,0,3,103,일반국도11호선,0,0,70.0,0.0,0.0,0.0,0,아라주공아파트,33.473494,126.545647,없음,인다마을,33.471061,126.545467,없음,2022,8,18,3
4,TEST_000004,2022-08-10,수,17,0,3,106,번영로,0,0,70.0,0.0,0.0,0.0,0,부록교 시종점,33.501477,126.569223,없음,봉개교 시종점,33.496863,126.581230,없음,2022,8,10,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291236,TEST_291236,2022-08-27,토,5,0,1,103,일반국도16호선,0,0,80.0,0.0,0.0,0.0,0,월산마을,33.472819,126.463030,없음,도근교,33.470483,126.460846,없음,2022,8,27,3
291237,TEST_291237,2022-08-19,금,20,0,2,103,일반국도11호선,0,0,60.0,0.0,0.0,0.0,0,영주교,33.305359,126.598914,없음,하례입구삼거리,33.300796,126.600332,없음,2022,8,19,3
291238,TEST_291238,2022-08-05,금,11,0,1,107,신대로,0,0,30.0,0.0,0.0,0.0,0,마리나사거리,33.493624,126.496769,있음,칠오광장사거리,33.497500,126.496946,있음,2022,8,5,3
291239,TEST_291239,2022-08-12,금,7,0,2,107,경찰로,0,0,60.0,0.0,0.0,0.0,0,국민의료보험관리공단,33.254782,126.507014,없음,의료보험조합,33.255659,126.507333,없음,2022,8,12,3


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701217 entries, 0 to 4701216
Data columns (total 28 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   id                     object        
 1   base_date              datetime64[ns]
 2   day_of_week            object        
 3   base_hour              int64         
 4   road_in_use            int64         
 5   lane_count             int64         
 6   road_rating            int64         
 7   road_name              object        
 8   multi_linked           int64         
 9   connect_code           int64         
 10  maximum_speed_limit    float64       
 11  vehicle_restricted     float64       
 12  weight_restricted      float64       
 13  height_restricted      float64       
 14  road_type              int64         
 15  start_node_name        object        
 16  start_latitude         float64       
 17  start_longitude        float64       
 18  start_turn_restricted 

In [10]:
# from sklearn.ensemble import RandomForestRegressor
# X_train = train.drop(['id', 'base_date', 'target'], axis = 1)
# y_train = train['target'].copy()
# baseline_model = RandomForestRegressor(random_state = 42, verbose = 3)
# baseline_model.fit(X_train, y_train)

from catboost import CatBoostRegressor
X_train = train.drop(['id', 'base_date', 'target'], axis = 1)
y_train = train['target'].copy()
cat_features = ['day_of_week', 'road_name', 'start_node_name', 'start_turn_restricted', 'end_node_name', 'end_turn_restricted']

baseline_model = CatBoostRegressor(random_state = 42, cat_features=cat_features)
baseline_model.fit(X_train, y_train)

Learning rate set to 0.155736
0:	learn: 14.2636701	total: 1.69s	remaining: 28m 8s
1:	learn: 12.8755621	total: 3.2s	remaining: 26m 39s
2:	learn: 11.7695406	total: 4.4s	remaining: 24m 21s
3:	learn: 10.9091615	total: 5.33s	remaining: 22m 6s
4:	learn: 10.2418286	total: 6.47s	remaining: 21m 28s
5:	learn: 9.6181847	total: 7.61s	remaining: 21m
6:	learn: 9.1090797	total: 8.4s	remaining: 19m 51s
7:	learn: 8.7287532	total: 9.18s	remaining: 18m 57s
8:	learn: 8.4220769	total: 10.3s	remaining: 18m 58s
9:	learn: 8.1874008	total: 11.3s	remaining: 18m 42s
10:	learn: 8.0018436	total: 12.6s	remaining: 18m 55s
11:	learn: 7.8397056	total: 13.6s	remaining: 18m 36s
12:	learn: 7.6891874	total: 14.4s	remaining: 18m 15s
13:	learn: 7.5654120	total: 15.5s	remaining: 18m 13s
14:	learn: 7.4714412	total: 16.5s	remaining: 18m 5s
15:	learn: 7.4022335	total: 17.1s	remaining: 17m 31s
16:	learn: 7.3416034	total: 18.3s	remaining: 17m 38s
17:	learn: 7.2852806	total: 19s	remaining: 17m 18s
18:	learn: 7.2387018	total: 20s	r

153:	learn: 5.8690168	total: 3m 8s	remaining: 17m 15s
154:	learn: 5.8678517	total: 3m 9s	remaining: 17m 15s
155:	learn: 5.8637170	total: 3m 10s	remaining: 17m 12s
156:	learn: 5.8597617	total: 3m 12s	remaining: 17m 11s
157:	learn: 5.8555301	total: 3m 13s	remaining: 17m 10s
158:	learn: 5.8523069	total: 3m 14s	remaining: 17m 9s
159:	learn: 5.8493298	total: 3m 15s	remaining: 17m 8s
160:	learn: 5.8432649	total: 3m 17s	remaining: 17m 6s
161:	learn: 5.8405630	total: 3m 18s	remaining: 17m 5s
162:	learn: 5.8373807	total: 3m 19s	remaining: 17m 3s
163:	learn: 5.8350383	total: 3m 20s	remaining: 17m 3s
164:	learn: 5.8319886	total: 3m 22s	remaining: 17m 2s
165:	learn: 5.8280327	total: 3m 23s	remaining: 17m
166:	learn: 5.8262409	total: 3m 24s	remaining: 16m 58s
167:	learn: 5.8226035	total: 3m 25s	remaining: 16m 56s
168:	learn: 5.8196882	total: 3m 26s	remaining: 16m 55s
169:	learn: 5.8160254	total: 3m 27s	remaining: 16m 54s
170:	learn: 5.8133748	total: 3m 29s	remaining: 16m 54s
171:	learn: 5.8105858	t

304:	learn: 5.5006058	total: 6m 20s	remaining: 14m 26s
305:	learn: 5.4978432	total: 6m 21s	remaining: 14m 25s
306:	learn: 5.4968733	total: 6m 22s	remaining: 14m 24s
307:	learn: 5.4952455	total: 6m 24s	remaining: 14m 23s
308:	learn: 5.4933527	total: 6m 25s	remaining: 14m 22s
309:	learn: 5.4917832	total: 6m 26s	remaining: 14m 21s
310:	learn: 5.4895723	total: 6m 28s	remaining: 14m 20s
311:	learn: 5.4878940	total: 6m 29s	remaining: 14m 18s
312:	learn: 5.4858175	total: 6m 31s	remaining: 14m 18s
313:	learn: 5.4839483	total: 6m 32s	remaining: 14m 17s
314:	learn: 5.4823870	total: 6m 33s	remaining: 14m 16s
315:	learn: 5.4815005	total: 6m 35s	remaining: 14m 15s
316:	learn: 5.4798286	total: 6m 36s	remaining: 14m 14s
317:	learn: 5.4790970	total: 6m 37s	remaining: 14m 12s
318:	learn: 5.4778715	total: 6m 38s	remaining: 14m 11s
319:	learn: 5.4769685	total: 6m 40s	remaining: 14m 10s
320:	learn: 5.4758538	total: 6m 41s	remaining: 14m 9s
321:	learn: 5.4741493	total: 6m 42s	remaining: 14m 7s
322:	learn: 

455:	learn: 5.3024121	total: 9m 35s	remaining: 11m 26s
456:	learn: 5.3018031	total: 9m 36s	remaining: 11m 25s
457:	learn: 5.3013658	total: 9m 37s	remaining: 11m 23s
458:	learn: 5.3003444	total: 9m 39s	remaining: 11m 22s
459:	learn: 5.2997693	total: 9m 40s	remaining: 11m 21s
460:	learn: 5.2993326	total: 9m 41s	remaining: 11m 20s
461:	learn: 5.2984994	total: 9m 43s	remaining: 11m 19s
462:	learn: 5.2976460	total: 9m 44s	remaining: 11m 18s
463:	learn: 5.2967985	total: 9m 45s	remaining: 11m 16s
464:	learn: 5.2953419	total: 9m 47s	remaining: 11m 15s
465:	learn: 5.2943093	total: 9m 48s	remaining: 11m 14s
466:	learn: 5.2936950	total: 9m 49s	remaining: 11m 13s
467:	learn: 5.2927901	total: 9m 51s	remaining: 11m 12s
468:	learn: 5.2907702	total: 9m 52s	remaining: 11m 10s
469:	learn: 5.2899299	total: 9m 53s	remaining: 11m 9s
470:	learn: 5.2893283	total: 9m 55s	remaining: 11m 8s
471:	learn: 5.2888068	total: 9m 56s	remaining: 11m 7s
472:	learn: 5.2877033	total: 9m 57s	remaining: 11m 5s
473:	learn: 5.

605:	learn: 5.1752899	total: 12m 48s	remaining: 8m 19s
606:	learn: 5.1740361	total: 12m 49s	remaining: 8m 18s
607:	learn: 5.1733789	total: 12m 50s	remaining: 8m 17s
608:	learn: 5.1724118	total: 12m 52s	remaining: 8m 15s
609:	learn: 5.1720008	total: 12m 53s	remaining: 8m 14s
610:	learn: 5.1701541	total: 12m 54s	remaining: 8m 13s
611:	learn: 5.1694651	total: 12m 56s	remaining: 8m 12s
612:	learn: 5.1689795	total: 12m 57s	remaining: 8m 10s
613:	learn: 5.1682762	total: 12m 58s	remaining: 8m 9s
614:	learn: 5.1674144	total: 12m 59s	remaining: 8m 8s
615:	learn: 5.1671763	total: 13m 1s	remaining: 8m 6s
616:	learn: 5.1663950	total: 13m 2s	remaining: 8m 5s
617:	learn: 5.1657321	total: 13m 3s	remaining: 8m 4s
618:	learn: 5.1646441	total: 13m 5s	remaining: 8m 3s
619:	learn: 5.1630078	total: 13m 6s	remaining: 8m 1s
620:	learn: 5.1625307	total: 13m 7s	remaining: 8m
621:	learn: 5.1618462	total: 13m 8s	remaining: 7m 59s
622:	learn: 5.1613520	total: 13m 10s	remaining: 7m 58s
623:	learn: 5.1605097	total:

756:	learn: 5.0791715	total: 16m 8s	remaining: 5m 10s
757:	learn: 5.0781084	total: 16m 9s	remaining: 5m 9s
758:	learn: 5.0778714	total: 16m 10s	remaining: 5m 8s
759:	learn: 5.0770767	total: 16m 12s	remaining: 5m 6s
760:	learn: 5.0768481	total: 16m 13s	remaining: 5m 5s
761:	learn: 5.0763955	total: 16m 14s	remaining: 5m 4s
762:	learn: 5.0760729	total: 16m 16s	remaining: 5m 3s
763:	learn: 5.0750448	total: 16m 17s	remaining: 5m 1s
764:	learn: 5.0745176	total: 16m 18s	remaining: 5m
765:	learn: 5.0741580	total: 16m 19s	remaining: 4m 59s
766:	learn: 5.0727257	total: 16m 21s	remaining: 4m 58s
767:	learn: 5.0721292	total: 16m 22s	remaining: 4m 56s
768:	learn: 5.0714484	total: 16m 23s	remaining: 4m 55s
769:	learn: 5.0710311	total: 16m 24s	remaining: 4m 54s
770:	learn: 5.0705755	total: 16m 26s	remaining: 4m 52s
771:	learn: 5.0700048	total: 16m 27s	remaining: 4m 51s
772:	learn: 5.0693245	total: 16m 28s	remaining: 4m 50s
773:	learn: 5.0688512	total: 16m 30s	remaining: 4m 49s
774:	learn: 5.0680333	t

907:	learn: 5.0036375	total: 19m 23s	remaining: 1m 57s
908:	learn: 5.0032758	total: 19m 25s	remaining: 1m 56s
909:	learn: 5.0027534	total: 19m 26s	remaining: 1m 55s
910:	learn: 5.0025039	total: 19m 27s	remaining: 1m 54s
911:	learn: 5.0022551	total: 19m 29s	remaining: 1m 52s
912:	learn: 5.0011668	total: 19m 30s	remaining: 1m 51s
913:	learn: 5.0008390	total: 19m 31s	remaining: 1m 50s
914:	learn: 5.0004023	total: 19m 33s	remaining: 1m 48s
915:	learn: 5.0002389	total: 19m 34s	remaining: 1m 47s
916:	learn: 4.9997675	total: 19m 35s	remaining: 1m 46s
917:	learn: 4.9994774	total: 19m 36s	remaining: 1m 45s
918:	learn: 4.9989677	total: 19m 38s	remaining: 1m 43s
919:	learn: 4.9986619	total: 19m 39s	remaining: 1m 42s
920:	learn: 4.9983804	total: 19m 40s	remaining: 1m 41s
921:	learn: 4.9981039	total: 19m 41s	remaining: 1m 39s
922:	learn: 4.9977161	total: 19m 43s	remaining: 1m 38s
923:	learn: 4.9972346	total: 19m 44s	remaining: 1m 37s
924:	learn: 4.9970333	total: 19m 45s	remaining: 1m 36s
925:	learn

In [11]:
test_model = test.drop(['id', 'base_date'], axis = 1)
pred = baseline_model.predict(test_model)

In [12]:
pred

array([23.81937647, 44.80958996, 62.69036661, ..., 22.52129778,
       23.85459519, 45.95965392])

In [13]:
output = pd.DataFrame({'id' : test.id, 'target':pred})
output.to_csv('baseline_catboost.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
